In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import ADASYN
from imblearn.pipeline import Pipeline as IMBPipeline

# Carga del nuevo conjunto de datos
data = pd.read_csv('/home/unaula/Descargas/training_sample.csv')

# Muestra las primeras filas del dataset para familiarizarte con los datos
print(data.head())
print(data.info())

# Verificación de la distribución de la variable objetivo
print(data['ordered'].value_counts())

# Selección de características y columna objetivo
features = [
    'basket_icon_click', 'basket_add_list', 'basket_add_detail', 'sort_by', 'image_picker',
    'account_page_click', 'promo_banner_click', 'detail_wishlist_add', 'list_size_dropdown',
    'closed_minibasket_click', 'checked_delivery_detail', 'checked_returns_detail', 'sign_in',
    'saw_checkout', 'saw_sizecharts', 'saw_delivery', 'saw_account_upgrade', 'saw_homepage',
    'device_mobile', 'device_computer', 'device_tablet', 'returning_user', 'loc_uk'
]
target = 'ordered'

# División en entrenamiento y prueba con estratificación
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Revisión de la distribución de la variable objetivo en el conjunto de entrenamiento
print(y_train.value_counts())

# Preprocesamiento
numeric_features = features  # Todas las características seleccionadas son numéricas en este caso
categorical_features = []  # No hay características categóricas en este caso

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features)
    ])

# Modelo XGBoost
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Pipeline con ADASYN para el balanceo de clases
pipeline = IMBPipeline([
    ('preprocessor', preprocessor),
    ('sampling', ADASYN(random_state=42)),
    ('model', model)
])

# Hiperparámetros para la búsqueda en grilla
param_grid = {
    'model__n_estimators': [200, 209, 220],
    'model__max_depth': [3, 4, 6],
    'model__learning_rate': [0.04, 0.048, 0.05],
    'model__subsample': [0.9, 1.0],
    'model__colsample_bytree': [0.5, 0.52, 0.55],
    'model__min_child_weight': [1, 2, 3]
}

# Búsqueda en grilla
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mejores hiperparámetros y evaluación del modelo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Mejores hiperparámetros:", grid_search.best_params_)
print("Precisión:", accuracy)
print("Informe de clasificación:\n", classification_rep)


                                UserID  basket_icon_click  basket_add_list  \
0  a720-6b732349-a720-4862-bd21-644732                  0                0   
1  a0c0-6b73247c-a0c0-4bd9-8baa-797356                  0                0   
2  86a8-6b735c67-86a8-407b-ba24-333055                  0                0   
3  6a3d-6b736346-6a3d-4085-934b-396834                  0                0   
4  b74a-6b737717-b74a-45c3-8c6a-421140                  0                1   

   basket_add_detail  sort_by  image_picker  account_page_click  \
0                  0        0             0                   0   
1                  0        0             0                   0   
2                  0        0             0                   0   
3                  0        0             0                   0   
4                  0        1             0                   0   

   promo_banner_click  detail_wishlist_add  list_size_dropdown  ...  \
0                   0                    0               